In [7]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [8]:
# tensor_dir = "../experiments_football/tensors/"
tensor_dir = "./test_tensors/yolov3_tiny_direct/layer8/"
output_dir = "./measurements/diff/"
layer = 8

test_tensors = os.listdir(tensor_dir)

In [9]:
def sort_fun(tensor_name):
    return int(tensor_name.split("_")[2].split(".")[0])

test_tensors.sort(key=sort_fun)

In [10]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]
tensor_decomposability=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [11]:
for i in range(10):
    print("************* Testing tensor: "+str(i)+"***************")
    split_layer.append(layer)
    tensor1 = torch.load(tensor_dir+test_tensors[i])
    tensor2 = torch.load(tensor_dir+test_tensors[i+1])
    tensor = tensor2 - tensor1
    tensor = tensor * (abs(tensor)>0.02)
    tensor = abs(tensor[0])


     # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank.append(calculate_slice_avg_rank(tensor))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(get_tensor_pictoriality(tensor))
    tensor_regularity.append(get_tensor_regularity(tensor))
    tensor_decomposability.append(get_tensor_decomposability(tensor))

    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1],tensor_decomposability[-1]))

    ## Jpeg compression
    try:
        print("Doing JPEG compression")
        normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
        reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
        ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
        snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
        mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
        jpeg_ratio.append(ratio_jpeg)
        jpeg_snr.append(snr_jpeg)
        jpeg_mse.append(mse_jpeg)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))
    except:
        jpeg_ratio.append(-1)
        jpeg_snr.append(-1)
        jpeg_mse.append(-1)

    try:
        print("Doing Decomposition compression")
        factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
        reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
        snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
        mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
        ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
        decomposition_ratio.append(ratio_decomposition)
        decomposition_snr.append(snr_decomposition)
        decomposition_mse.append(mse_decomposition)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))
    except:
        decomposition_ratio.append(-1)
        decomposition_snr.append(-1)
        decomposition_mse.append(-1)

    try:
        print("Doing Regression compression")
        factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
        reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
        snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
        mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
        ratio_regression = tensor.numel()*4 / compressed_size_regression
        regression_ratio.append(ratio_regression)
        regression_snr.append(snr_regression)
        regression_mse.append(mse_regression)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))
    except:
        regression_ratio.append(-1)
        regression_snr.append(-1)
        regression_mse.append(-1)

    try:
        print("Doing SketchML compression")
        minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
        reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
        snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
        mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
        ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
        sketchml_ratio.append(ratio_sketchml)
        sketchml_snr.append(snr_sketchml)
        sketchml_mse.append(mse_sketchml)
        print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))
    except:
        sketchml_ratio.append(-1)
        sketchml_snr.append(-1)
        sketchml_mse.append(-1)


************* Testing tensor: 0***************
Rank: 14.296875, Sparsity: 0.900356, Picoriality: 0.725458, Regularity: 0.502741, Decomposability: -0.099760
Doing JPEG compression
Ratio: 5.376581, SNR: 25.761187, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.909290, SNR: 135.739355, MSE: 0.000000
Doing Regression compression
Ratio: 16.484664, SNR: 9.876090, MSE: 0.000014
Doing SketchML compression
Ratio: 31.914431, SNR: 8.376811, MSE: 0.000020
************* Testing tensor: 1***************


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Rank: 12.851562, Sparsity: 0.916917, Picoriality: 0.723161, Regularity: 0.508701, Decomposability: 0.011418
Doing JPEG compression
Ratio: 5.598253, SNR: 24.843535, MSE: 0.000000
Doing Decomposition compression
Ratio: 1.011550, SNR: 136.064625, MSE: 0.000000
Doing Regression compression
Ratio: 16.628009, SNR: 9.920002, MSE: 0.000011
Doing SketchML compression
Ratio: 36.223129, SNR: 8.269746, MSE: 0.000016
************* Testing tensor: 2***************


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Rank: 11.187500, Sparsity: 0.935651, Picoriality: 0.702720, Regularity: 0.514000, Decomposability: 0.139423
Doing JPEG compression
Ratio: 6.012333, SNR: 23.525262, MSE: 0.000000
Doing Decomposition compression
Ratio: 1.162011, SNR: 136.966038, MSE: 0.000000
Doing Regression compression
Ratio: 17.073402, SNR: 10.223963, MSE: 0.000008
Doing SketchML compression
Ratio: 42.777407, SNR: 8.454603, MSE: 0.000012
************* Testing tensor: 3***************


...

Rank: 12.414062, Sparsity: 0.918373, Picoriality: 0.709281, Regularity: 0.498533, Decomposability: 0.045072
Doing JPEG compression
Ratio: 5.167393, SNR: 20.020669, MSE: 0.000001
Doing Decomposition compression
Ratio: 1.047199, SNR: 136.426134, MSE: 0.000000
Doing Regression compression
Ratio: 16.628009, SNR: 9.939827, MSE: 0.000012
Doing SketchML compression
Ratio: 36.683837, SNR: 7.875245, MSE: 0.000019
************* Testing tensor: 8***************


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)
/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Rank: 14.601562, Sparsity: 0.901315, Picoriality: 0.722550, Regularity: 0.493842, Decomposability: -0.123197
Doing JPEG compression
Ratio: 4.532516, SNR: 19.371459, MSE: 0.000002
Doing Decomposition compression
Ratio: 0.890316, SNR: 135.651321, MSE: 0.000000
Doing Regression compression
Ratio: 16.628009, SNR: 9.834267, MSE: 0.000014
Doing SketchML compression
Ratio: 32.145630, SNR: 8.447702, MSE: 0.000019
************* Testing tensor: 9***************


/home/rex/gitRepo/split_DNN_framework/characteristics/lib/regression.py:20: RankWarning: Polyfit may be poorly conditioned
  m = np.polyfit(x,y,polynominal)


Rank: 14.804688, Sparsity: 0.912179, Picoriality: 0.697582, Regularity: 0.474289, Decomposability: -0.138822
Doing JPEG compression
Ratio: 4.923007, SNR: 19.702418, MSE: 0.000001
Doing Decomposition compression
Ratio: 0.878100, SNR: 135.935431, MSE: 0.000000
Doing Regression compression
Ratio: 16.773868, SNR: 9.205786, MSE: 0.000015
Doing SketchML compression
Ratio: 34.907917, SNR: 7.709471, MSE: 0.000021


In [12]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [13]:
df.to_csv(output_dir+"layer"+str(layer)+"_diff.csv",index=False, header=True)